In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
import sqlite3

# Database setup
def setup_database():
    conn = sqlite3.connect('inventory.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS inventory (
                        id INTEGER PRIMARY KEY,
                        product_name TEXT NOT NULL,
                        quantity INTEGER NOT NULL,
                        price_per_unit REAL NOT NULL)''')
    conn.commit()
    conn.close()

# Add product to the inventory
def add_product():
    product_name = entry_product_name.get()
    quantity = entry_quantity.get()
    price = entry_price.get()
    
    if not product_name or not quantity.isdigit() or not price:
        messagebox.showwarning("Input Error", "Please enter valid product details!")
        return

    conn = sqlite3.connect('inventory.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO inventory (product_name, quantity, price_per_unit) VALUES (?, ?, ?)",
                   (product_name, int(quantity), float(price)))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", f"Added {product_name} to inventory.")
    entry_product_name.delete(0, tk.END)
    entry_quantity.delete(0, tk.END)
    entry_price.delete(0, tk.END)
    view_inventory()

# View inventory items
def view_inventory():
    tree.delete(*tree.get_children())
    conn = sqlite3.connect('inventory.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM inventory")
    rows = cursor.fetchall()
    conn.close()
    for row in rows:
        tree.insert('', tk.END, values=row)

# Update product quantity
def update_quantity():
    selected_item = tree.focus()
    if not selected_item:
        messagebox.showwarning("Selection Error", "Please select an item to update!")
        return
    
    new_quantity = entry_quantity.get()
    if not new_quantity.isdigit():
        messagebox.showwarning("Input Error", "Please enter a valid quantity!")
        return
    
    item_id = tree.item(selected_item)['values'][0]
    conn = sqlite3.connect('inventory.db')
    cursor = conn.cursor()
    cursor.execute("UPDATE inventory SET quantity = ? WHERE id = ?", (int(new_quantity), item_id))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Quantity updated.")
    view_inventory()

# Delete a product
def delete_product():
    selected_item = tree.focus()
    if not selected_item:
        messagebox.showwarning("Selection Error", "Please select an item to delete!")
        return
    
    item_id = tree.item(selected_item)['values'][0]
    conn = sqlite3.connect('inventory.db')
    cursor = conn.cursor()
    cursor.execute("DELETE FROM inventory WHERE id = ?", (item_id,))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Product deleted.")
    view_inventory()

# Main GUI
root = tk.Tk()
root.title("Inventory Management System")
root.geometry("600x400")

setup_database()

# Input fields
frame_input = tk.Frame(root)
frame_input.pack(pady=10)

tk.Label(frame_input, text="Product Name").grid(row=0, column=0, padx=5)
entry_product_name = tk.Entry(frame_input)
entry_product_name.grid(row=0, column=1)

tk.Label(frame_input, text="Quantity").grid(row=1, column=0, padx=5)
entry_quantity = tk.Entry(frame_input)
entry_quantity.grid(row=1, column=1)

tk.Label(frame_input, text="Price per Unit").grid(row=2, column=0, padx=5)
entry_price = tk.Entry(frame_input)
entry_price.grid(row=2, column=1)

# Buttons
frame_buttons = tk.Frame(root)
frame_buttons.pack(pady=10)

tk.Button(frame_buttons, text="Add Product", command=add_product).grid(row=0, column=0, padx=5)
tk.Button(frame_buttons, text="Update Quantity", command=update_quantity).grid(row=0, column=1, padx=5)
tk.Button(frame_buttons, text="Delete Product", command=delete_product).grid(row=0, column=2, padx=5)

# Inventory table
tree = ttk.Treeview(root, columns=('ID', 'Product Name', 'Quantity', 'Price per Unit'), show='headings')
tree.heading('ID', text='ID')
tree.heading('Product Name', text='Product Name')
tree.heading('Quantity', text='Quantity')
tree.heading('Price per Unit', text='Price per Unit')
tree.pack(fill=tk.BOTH, expand=True)

view_inventory()

root.mainloop()